## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откроем файл с данными и изучим общую информацию. 

In [1]:
import pandas as pd
solvency_stat = pd.read_csv('/datasets/data.csv')
solvency_stat


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


#### Изучим общую информацию по таблице и проверим пропуски

In [2]:
solvency_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
solvency_stat.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

#### Выведем основную информацию по датафрейму

In [4]:
solvency_stat.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


##### Наблюдения после выборки данных:
##### 1. Отсутствуют значения в столбцах 'days_employed' и 'total_income' (а это около 10% от общего числа клиентов). Необходимо будет проверить, там где отсутствуют данные по обоим столбцам это одни и те же строки или нет.
##### 2. Минимальное значение в столбце 'children'  -1.  Возможно заемщик отказался сообщать информацию или ее не запрашивали, что очень странно для кредитной анкеты. Значит можно предположить, что просто таким образом записано количество детей '1'.
##### Максимальное значение в столбце 'children'  20. Тоже маловероятно. Вероятно проблема такая же как и с минимальным значением, только в данном случае количество детей скорее всего равняется 2.
##### Отрицательные значения по общему трудовому стажу 'days_employed'. Значит где-то данные некорректны. 
##### Среднее значение по количеству отработанных дней 'days_employed'  63046 дней. Это 172 с лишним года. Тоже нужно искать проблему.
##### Минимальный возраст  'dob_years' -- 0. Возможно просто где-то не указан (что тоже странно для кредитной анкеты).

In [5]:
# Проверим данные по трудовому стажу и по ежемесячному доходу
solvency_stat[(solvency_stat['total_income'].isnull() == True) & (solvency_stat['days_employed'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


##### Предположение оказалось верным: в строках, где отсутствуют данные по трудовому стажу, отсутствуют данные и по ежемесячному доходу.
##### Проверим по типу занятости, возможно это один тип

In [6]:
solvency_stat[(solvency_stat['total_income'].isnull() == True) & (solvency_stat['days_employed'].isnull() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

##### Нет, тип занятости разный. Значит взять среднее по 'income_type' нельзя.


#### Выше мы обнаружили странные данные в столбцах 'children', 'days_employed' и 'dob_years'. Проверим их на корректность.

In [7]:
#Разберёмся со странным количеством детей
print("Количество семей с -1 ребенком:", solvency_stat[solvency_stat['children'] == -1].count()[0])
print("Количество семей с 20 детьми:", solvency_stat[solvency_stat['children'] == 20].count()[0])
#Проверим семьи с 20 детьми на уникальность
print("Количество уникальных людей с 20 детьми:", len(solvency_stat[solvency_stat['children'] == 20]['total_income'].unique()))

Количество семей с -1 ребенком: 47
Количество семей с 20 детьми: 76
Количество уникальных людей с 20 детьми: 68


Предположение по -1 ребёнку подтвердилось -- это ошибка.
По 20 детям в семье тоже -- данные не уникальны, а значит это тоже ошибка.

In [8]:
#Разберёмся с возрастом равным 0
print("Количество людей с возрастом равным 0:", solvency_stat[solvency_stat['dob_years'] == 0].count()[0])

Количество людей с возрастом равным 0: 101


Данные отсутствуют у слишком большого количества клиентов. Скорее всего это ошибка.

In [9]:
#Разберёмся со стажем (отрицательные данные и слишком большое число отработанных дней)
print("Строк со стажем > 0:", solvency_stat[solvency_stat['days_employed'] > 0].shape[0])
print("Строк со стажем < 0:", solvency_stat[solvency_stat['days_employed'] < 0].shape[0])
#Посмотрим сколько из них пенсионеров
pens = solvency_stat[(solvency_stat['days_employed'] > 0) & (solvency_stat['income_type'] == 'пенсионер')]
print("Количество пенсионеров со стажем > 0:", pens.shape[0])
#Пенсионеров большинство, выведем среднее количество отработанных ими дней
print("Отработанных пенсионерами дней в среднем", pens['days_employed'].mean())

Строк со стажем > 0: 3445
Строк со стажем < 0: 15906
Количество пенсионеров со стажем > 0: 3443
Отработанных пенсионерами дней в среднем 365003.4912448612


Большинство значений по стажу отрицательные и не относятся к пенсионерам.
Значения >0 за исключением 2-х человек -- это пенсионеры.
Среднее количество отработанных пенсионерами лет в районе 1000. Такого быть не может.

### Вывод:
##### 1.В файле 'data.csv' обнаружен массив из данных о 21525 клиентах.
##### Существуют пропущенные значения по общему трудовому стажу и ежемесячному доходу у 2174 клиентов. 
##### Значения уровня образования указаны в разных регистрах. Необходимо все значения привести к нижнему регистру.

##### 2.Странностей в ежемесячных доходах не выявлено, если не считать пропущенные значения.

##### 3.В данные о возрасте клиентов попали нулевые значения, их надо устранить.

##### 4.У большинства клиентов численность детей варьируется от 0 до 4, однако, у некоторых детей -1 чел. или 20. Заменим эти данные из предположения, что вместо 1 ребенка записали вручную -1, а под 20 детьми подразумевали только двоих.

##### 5.Самые странные данные в днях трудового стажа клиентов -- присутствуют отрицательные и положительные значения (но длительностью в сотни лет). Неплохо бы запросить информацию от выгружавших эти данные, чтобы выяснить именно под ними подразумевается. НО для ответов на запрашиваемые в проекте вопросы нам эти данные не требуются. Поэтому просто заполним пропуски в столбце медианными значениями.

## Шаг 2. Предобработка данных

### Обработка пропусков

1.Заменим пустые значения в датафрейме медианными.
2.Значения уровня образования указаны в разных регистрах. Приведём их к нижнему регистру.
3.Уберём нулевые значения в данных о возрасте клиентов, заменив на средние.

In [10]:
#1
#заменяем отрицательные значения стажа работы
solvency_stat['days_employed']= abs(solvency_stat['days_employed'])


#выводим медиану столбцов с пропущенными значениями:
days_employed_median = solvency_stat['days_employed'].median() 
total_income_median = solvency_stat['total_income'].median()


#заполняем пропуски значениями их медиан
solvency_stat['days_employed']= solvency_stat['days_employed'].fillna(value=days_employed_median)
solvency_stat['total_income']= solvency_stat['total_income'].fillna(value=total_income_median)

solvency_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [11]:
#2
# Столбец 'education' переведём в нижний регистр
solvency_stat['education'] = solvency_stat['education'].str.lower()

В выводах из шага 1 мы предположили, что вместо 1 ребенка записали вручную -1, а под 20 детьми подразумевали только двоих.
Поправим эти данные в соответствии с предположением.з

In [12]:
#заменим ошибочные значения в количестве детей из следующего предположения:
#вместо 20 детей укажем 2-х, вместо -1 укажем 1.
solvency_stat.loc[solvency_stat['children'] == 20, 'children'] = 2
solvency_stat.loc[solvency_stat['children'] == -1, 'children'] = 1

In [13]:
#3
#тип занятости клиентов с нулевым возрастом.
solvency_stat.loc[solvency_stat['dob_years'] == 0, 'income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

In [14]:
#найдем медианные значения возрастов по каждому типу занятости
age_median = solvency_stat.groupby('income_type')['dob_years'].median()
age_median

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

In [15]:
#заменим нулевые значения на медианы по каждому типу занятости
solvency_stat.loc[(solvency_stat['dob_years'] == 0) & (solvency_stat['income_type'] == 'сотрудник'), 'dob_years'] = age_median[6]
solvency_stat.loc[(solvency_stat['dob_years'] == 0) & (solvency_stat['income_type'] == 'пенсионер'), 'dob_years'] = age_median[4]
solvency_stat.loc[(solvency_stat['dob_years'] == 0) & (solvency_stat['income_type'] == 'компаньон'), 'dob_years'] = age_median[3]
solvency_stat.loc[(solvency_stat['dob_years'] == 0) & (solvency_stat['income_type'] == 'госслужащий'), 'dob_years'] = age_median[2]

In [16]:
#проверим сколько строк с нулями теперь
solvency_stat[solvency_stat['dob_years'] == 0].count()[0]

0

Строки с возрастом 0 отсутствуют.

In [17]:
#взглянем на таблицу
solvency_stat.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,60378.032733,43.496167,0.817236,0.972544,0.080883,1.651595e+05
std,0.755528,133257.558514,12.231538,0.548138,1.420324,0.272661,9.786607e+04
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.608174,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2194.220567,43.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,4779.587738,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод
#### Все строки теперь заполнены и приведены в должный вид
#### 1.Заменили пустые значения в датафрейме медианными. 
#### 2.Привели значения образования к нижнему регистру. 
#### 3.Поправили данные в количестве детей (-1 заменили на 1, 20 заменили на 2)
#### 4.Убрали нулевые значения в данных о возрасте клиентов, заменив на медианы.

### Замена типа данных

Посмотрим еще раз общую информацию по массиву данных.

In [18]:
solvency_stat.dtypes

children              int64
days_employed       float64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

На данный момент возраст клиентов и доход записаны вещественным числом, изменим тип значений с вещественного на целый.

In [19]:
#используем метод 'astype', так как мы уже обработали пропущенные значения (иначе можно было бы использовать 'to_numeric')
solvency_stat['dob_years'] = solvency_stat['dob_years'].astype('int')
solvency_stat['total_income'] = solvency_stat['total_income'].astype('int')
solvency_stat.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int64
purpose              object
dtype: object

### Вывод
#### Все данные теперь в удобном формате. Месячную зарплату и возраст клиентов перевели в 'int', чтобы видеть целочисленные значения. Как уже описывалось выше, для ответов на запрашиваемые в проекте вопросы столбец со стажем нам не требуются - оставили вещественный тип данных.

### Обработка дубликатов

Посмотрим на количество дубликатов

In [20]:
solvency_stat.duplicated().sum()

71

In [21]:
solvency_stat[solvency_stat.duplicated(keep=False)].sort_values(by=['total_income', 'dob_years'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8853,1,2194.220567,23,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу
15892,0,2194.220567,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем
19321,0,2194.220567,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем
20297,1,2194.220567,23,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу
3452,0,2194.220567,29,высшее,0,женат / замужем,0,M,сотрудник,0,145017,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
5865,0,2194.220567,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145017,операции со своей недвижимостью
9528,0,2194.220567,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145017,операции со своей недвижимостью
6537,0,2194.220567,71,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
7938,0,2194.220567,71,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы


In [22]:
#некоторые строки выглядят как дублирование данных
#уберём их
solvency_stat = solvency_stat.drop_duplicates()
#и посмотрим на результат
solvency_stat.duplicated().sum()

0

### Вывод
#### Для подобных данных логичнее было бы иметь столбец с ID заёмщика. Но поскольку в рамках проекта у нас такой столбец не предусмотрен, предположим, что полное совпадение всех остальных полей (пол, возраст, доход и так далее) невозможно
#### По итогу из датафрейма удалены дубликаты в количестве 71 шт.

### Лемматизация

В таблице есть столбец 'purpose' в котором содержатся похожие друг на друга цели получения кредита, но записанные разными словами. Для чистоты данных необходимо произвести лемматизацию этого столбца. 

In [23]:
#импортируем библиотеки pymystem и collections
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [24]:
solvency_stat['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [25]:
#ключевые слова после подсчёта кол-ва вхождений лемм можем выделить вручную
key_words = ['недвижимость', 'жилье', 'образование', 'автомобиль', 'свадьба']

# проведем лемматизацию
def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in key_words:
        if word in lemma:
            lemma = word
    return lemma

solvency_stat['purpose_category'] = solvency_stat['purpose'].apply(lemmatize) 
       
solvency_stat.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [26]:
solvency_stat['purpose_category'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_category, dtype: int64

In [27]:
solvency_stat.loc[solvency_stat['purpose_category'] == 'жилье', 'purpose_category'] = 'недвижимость'
solvency_stat.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод
#### Из цели получения кредита каждого клиента было выделено по одному ключевому слову

### Категоризация данных

Для ответа на поставленные задачей вопросы нам понадобится категоризация по возрасту, доходам и целям

In [28]:
#в качестве границ для категоризации возрастов воспользуемся кватилями по столбцу 'dob_years'
age_quantile = solvency_stat['dob_years'].describe()
age_quantile[4:7]

25%    33.0
50%    42.0
75%    53.0
Name: dob_years, dtype: float64

In [29]:
#напишем функцию, которая принимает на вход возраст клиента и возвращает возрастную категорию
def age_group(age):
    if age <= age_quantile[4]: return 'до 34'
    elif age_quantile[4] < age <= age_quantile[5]: return '34-43'
    elif age_quantile[5] < age <= age_quantile[6]: return '43-53'
    else: return '53+'
        
solvency_stat['age_category'] = solvency_stat['dob_years'].apply(age_group)




/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
#в качестве границ для категоризации доходов воспользуемся кватилями по столбцу 'total_income'
total_inc_quantile = solvency_stat['total_income'].describe()
total_inc_quantile[4:7]

25%    107623.00
50%    145017.00
75%    195813.25
Name: total_income, dtype: float64

In [31]:
def total_inc_group(income):
    if income <= total_inc_quantile[4]: return 'низкий'
    elif total_inc_quantile[4] < income <= total_inc_quantile[5]: return 'средний'
    elif total_inc_quantile[5] < income <= total_inc_quantile[6]: return 'выше среднего'
    else: return 'высокий'
    
solvency_stat['total_inc_category'] = solvency_stat['total_income'].apply(total_inc_group)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [32]:
#произведем категоризацию по наличию/отсутствию детей, где 1 - есть дети, 0 - нет
def children_group(children):
    if children > 0: return 1
    else: return 0

# добавим новый столбец с бинарным признаком в наш исходный массив
solvency_stat['children_category'] = solvency_stat['children'].apply(children_group)

#посмотрим на таблицу
solvency_stat.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,age_category,total_inc_category,children_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,34-43,высокий,1
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,34-43,средний,1
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,до 34,выше среднего,0
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,до 34,высокий,1
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,43-53,выше среднего,0


In [33]:
#посмотрим статистику по возрастам
solvency_stat['age_category'].value_counts()

43-53    5448
до 34    5366
34-43    5365
53+      5275
Name: age_category, dtype: int64

In [34]:
#посмотрим на статистику по доходам
solvency_stat['total_inc_category'].value_counts()

средний          6415
низкий           5364
высокий          5364
выше среднего    4311
Name: total_inc_category, dtype: int64

In [35]:
#категоризация по целям была произведена при лемматизации
#посмотрим на статистику по целям кредита
solvency_stat['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

In [36]:
solvency_stat['children_category'].value_counts()

0    14091
1     7363
Name: children_category, dtype: int64

### Вывод: 1. Основной возраст клиентов составляет 43-53 года
###                2. В основном у заёмщиков  средний доход
###                3. Подавляющее большинство берёт кредиты на покупку недвижимости
### 4. Заёмщиков без детей в 2 раза больше

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [37]:
# создадим сводную таблицу
data_pivot = solvency_stat.pivot_table(index = ['children_category'], values = 'debt').round(3)
#data_pivot['ratio'] = data_pivot[1] / data_pivot[0]
data_pivot.head()

,debt
children_category,
0,0.075
1,0.092


### Вывод
Заёмщики не имеющие детей менее склонны к просрочкам по кредитам

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [38]:
data_pivot = solvency_stat.pivot_table(index = ['family_status'], columns = 'debt', values = 'gender', aggfunc = 'count')

# посчитаем вероятность задолженности для каждого вида семейного положения
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)
data_pivot.sort_values('ratio', ascending = False)

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2536,274,0.098
гражданский брак,3763,388,0.093
женат / замужем,11408,931,0.075
в разводе,1110,85,0.071
вдовец / вдова,896,63,0.066


### Вывод
Заёмщики никогда не состоящие и никогда до этого не состоявшие в браке больше других склонны к просрочкам

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
data_pivot = solvency_stat.pivot_table(index = ['total_inc_category'], columns = 'debt', values = 'gender', aggfunc = 'count')

# посчитаем вероятность задолженности для каждой группы доходов
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)
data_pivot.sort_values('ratio', ascending = False)

debt,0,1,ratio
total_inc_category,,,
выше среднего,3927,384,0.089
средний,5868,547,0.085
низкий,4937,427,0.080
высокий,4981,383,0.071


### Вывод
Удивительно, но к просрочкам склонны клиенты с доходом выше среднего. Также странно, что следом идут клиенты не с низким доходом, а со средним. 
Наименее склонны к просрочкам клиенты с высоким уровнем дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [40]:
data_pivot = solvency_stat.pivot_table(index = ['purpose_category'], columns = 'debt', values = 'gender', aggfunc = 'count')

# посчитаем вероятность задолженности для каждой цели кредита
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)
data_pivot.sort_values('ratio', ascending = False)


debt,0,1,ratio
purpose_category,,,
автомобиль,3903,403,0.094
образование,3643,370,0.092
свадьба,2138,186,0.080
недвижимость,10029,782,0.072


### Вывод
Самые ответственные плательщики -- те, кто берёт кредит на покупку жилья. Хуже всего заёмщики возвращают кредиты автомобили и образование.

### Шаг 4. Общий вывод
### В целом, в результате проведённого исследования можно нарисовать портрет идеального заёмщика:
### Состоит (либо состоял) в официальных отношениях, не имеет детей, уровень дохода высокий (от 195 тыс.руб.), цель получения кредита -- покупка недвижимости.